### Extract the tweets from twitter and save in MySQL

In [19]:
# !pip3 install git+https://github.com/tweepy/tweepy.git
# !pip3 install tweepy==3.10

In [3]:
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 
import re
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import mysql.connector
import pandas as pd
from textblob import TextBlob

In [4]:
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy
# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude,retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False

In [5]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [6]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database="TwitterData",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()

In [7]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [8]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

Yes, Donald Trump's final days in office were even worse than we thought #SmartNews Taking everything personally, t… https://t.co/OgQbzQirK6
Long: None, Lati: None
@Cartref35 @ThinkEmily @SteveBakerHW And they've said why they are doing it and it pre dates the blm movement and t… https://t.co/59dDpSTgni
Long: None, Lati: None
RT @azzurri: Teammates. Friends. Colleagues. Champions. 💙

#Azzurri 🏆🇮🇹 #EURO2020 #VivoAzzurro https://t.co/KIIVAUGWM4
Long: None, Lati: None
RT @chiellini: 🦍 SuperQuark: Speciale #Euro2020 https://t.co/y2it7WasTl
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @AndrewBrownhil: I'm just going to leave this here ❤ #ENG #englandfans #EURO2020 https://t.co/TeQJ87XehW
Long: None, Lati: None
RT @brfootball: The official Team of the Tournament for #EURO2020 https://t.co/AMG3pPoosf
Long: None, Lati: None
@ReaganWorld America is in danger of becoming a 3rd world country under Democrat Anti

RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
Tuesday PM on https://t.co/TstiTuD8rx is live! We have a new Ben Garrison editorial cartoon ready for you! "Mad Max… https://t.co/FVNwZwBFTA
Long: None, Lati: None
RT @EURO2020: 🇮🇹 The moment Italy lifted their second EURO title! 🏆 

@azzurri | #ITA | #EURO2020 https://t.co/MVl5tjZoyK
Long: None, Lati: None
RT @goal: Copa America Golden Boot: Lionel Messi 🏅

#EURO2020 Golden Boot: Cristiano Ronaldo 🏅

They're from another planet 👽 https://t.co/…
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @ZubyMusic: 19 times as many people were killed during the BLM riots last year (19) than during the rush of The Capitol on Jan 6th (1).…
Long: None, Lati: None
RT @Ravagiing: LISTEN TO THIS! 

Everyone that chooses to side with the blm eff TERRORISTS targeting Indians! You are part of the F^ proble…
Long: None,

wait until you hear about Afro-Cubans
Long: None, Lati: None
RT @brfootball: The official Team of the Tournament for #EURO2020 https://t.co/AMG3pPoosf
Long: None, Lati: None
RT @AndrewBrownhil: I'm just going to leave this here ❤ #ENG #englandfans #EURO2020 https://t.co/TeQJ87XehW
Long: None, Lati: None
@shinyshannon73 @samstein An unarmed woman... If she was black, BLM would still be rioting!
Long: None, Lati: None
RT @angelomangiante: Champions League winner. 🏆

Final at #EURO2020

Potentially a winner of the Ballon d'Or. 

Maestro #Jorginho 🔥

#ITAES…
Long: None, Lati: None
My daughter won our family #EURO2020 sweepstake. Her prize... 😂 #winnerwinnerpastadinner https://t.co/ugreqQrjZk
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @MeidasJeremy19: Isn't it funny how Donald Trump felt the need to be down in a bunker during BLM protests, but when it was Nazis outside…
Long: None, Lati: None
@Guto_Harr

RT @ZubyMusic: 19 times as many people were killed during the BLM riots last year (19) than during the rush of The Capitol on Jan 6th (1).…
Long: None, Lati: None
RT @ysimpsonpower: Sandra Bland’s death made me realize that I wasn’t safe. A black woman stopped on a technicality, proclaiming her right…
Long: None, Lati: None
RT @NizaarKinsella: Jorginho on his Ballon d'Or contention: "We live for dreams and [the Ballon d’Or] hinges on what criteria they use. If…
Long: None, Lati: None
RT @AaronParnas: The First Amendment ensures that individuals have the ability to peacefully protest. 

Ron DeSantis criminalized lawful, p…
Long: None, Lati: None
RT @abcdemilyx_: My class wrote this today, I am so humbled and moved by their words. Please share to get it to the right people 💛 @MarcusR…
Long: None, Lati: None
RT @utdreport: Harry Maguire won 84% of his aerials duels at #EURO2020 (21/25), the best winning percentage in recorded European Championsh…
Long: None, Lati: None
RT @mdbuckley: This

RT @osee80: First time Ronaldo is not in top 11 of a tournament 

Ronaldo must be drinking coke now

#KeepItOn
#Messi 
#EndSARS 
Football…
Long: None, Lati: None
RT @abcdemilyx_: My class wrote this today, I am so humbled and moved by their words. Please share to get it to the right people 💛 @MarcusR…
Long: None, Lati: None
@gbengaadeosun_ @walter_walper @EURO2020 The defending champions were Portugal and yet didn't have home advantage.… https://t.co/fWiMdEwGML
Long: None, Lati: None
RT @caleonard: More awards! Congrats to @Jonmaesha for winning first place in student features reporting from @AZPressClub for this excelle…
Long: None, Lati: None
RT @thecasualultra: England fans setting off fireworks outside the Italy team hotel at 2AM 😂🏴󠁧󠁢󠁥󠁮󠁧󠁿 #Euro2020 https://t.co/JefiSmFMD9
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
L

...

RT @HafsaHalawa: I wonder if @Kaepernick7 has seen any of this, or is watching from across the Atlantic, and sees what his brave, lone effo…
Long: None, Lati: None
RT @HarryMaguire93: Proud #euro2020 🏴󠁧󠁢󠁥󠁮󠁧󠁿 https://t.co/HSccCbdOAJ
Long: None, Lati: None
RT @EURO2020: 🤯 Italy have not lost any of the 16 matches with Giorgio Chiellini and Leonardo Bonucci both in the starting XI under Roberto…
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @OrwellDream: @JoyAnnReid @RonDeSantisFL HB-1 was only for BLM protesters.
Long: None, Lati: None
@LBC @NickFerrariLBC BLM  must  be well pleased with how much racial shit is being stirred up  aided &amp; abetted by mainstream media..
Long: None, Lati: None
@Falco_UTD @lincsbirder2019 @StretfordPaddck It's not about life, nobody wanted to kill him, BLM was formed to prot… https://t.co/6OsB8gl8iW
Long: None, Lati: None
He’s not lying
Long: None, Lati: None
RT @ZubyMusi

RT @brfootball: The official Team of the Tournament for #EURO2020 https://t.co/AMG3pPoosf
Long: None, Lati: None
RT @peterjukes: Rebekah Brooks has felt the tide changing and told Harry Cole to tone down his ‘anti woke’ BLM=Marxism rhetoric. Don’t be f…
Long: None, Lati: None
RT @patrickcherry: Rocking https://t.co/VhhmAbQw7S
Long: None, Lati: None
RT @goal: Football press conferences can be boring...

But #EURO2020 brought us some brilliant moments 😂 https://t.co/PvgAZASXz3
Long: None, Lati: None
RT @azzurri: 🔝 @gigiodonna1, @bonucci_leo19, #Spinazzola, #Jorginho and @federicochiesa make UEFA's Team of the Tournament 👏

#Azzurri 🏆🇮🇹…
Long: None, Lati: None
RT @verbalese: Megyn Kelly - we watched Jan 6th live 

We saw it with our own eyes

No amount of gaslighting changes what we saw. 

It wasn…
Long: None, Lati: None
RT @Arron_banks: Would those be the footballers who praise BLM by taking the knee , who loot shops , deface statues of war leaders like Chu…
Long: None, Lati: None
RT @Jo

RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @brfootball: The official Team of the Tournament for #EURO2020 https://t.co/AMG3pPoosf
Long: None, Lati: None
@Mindyourslady @theejenbunny2 This mall has few security; maybe a handful &amp; they are all OLD or meek looking. Used… https://t.co/u55a5NZ5Rx
Long: None, Lati: None
@tedlieu You backed the burning of black communities because you are a racist psychopath. #BLM
Long: None, Lati: None
RT @goal: Premier League Golden Boot ✔️
La Liga Golden Boot ✔️
Serie A Golden Boot ✔️
Champions League Golden Boot ✔️
#EURO2020 Golden Boot…
Long: None, Lati: None
RT @Nicocantor1: Someone just forwarded this to me on WhatsApp:

Comparative stats between #Euro2020 and #CopaAmérica https://t.co/3j4z3kVP…
Long: None, Lati: None
Italy wins! 

It’s their first European Championship since 1968!

The amazing win symbolizes a renewed sense of pri… https://t.co/0LwOLmn6Vh
Long: None, Lati: None
Cana

I agree. Centre forward was Italy's weak link. Credit to them to have won despite Immobile's best efforts to not ma… https://t.co/mn1ZCynt1D
Long: None, Lati: None
RT @mdbuckley: This is incredible. 

#BLM #eng #ThreeLions https://t.co/s4hY1vaKfa
Long: None, Lati: None
RT @BarryLarkin6: @kim @sdvicto "How odd iQ45 needed to be down in his WH bunker during BLM protests, 
but when Nazis MAGAmoron feces smear…
Long: None, Lati: None
RT @Quicktake: LOOK: Artist @Akse_P19 has restored the mural of Marcus Rashford, which was defaced after England’s loss in the #EURO2020 fi…
Long: None, Lati: None
@NicoleFordTV @KDKA Good job Democrats.  You took the late 20th centurys greatest success and totally ruined it with ur @BLM nonsense
Long: None, Lati: None
RT @brfootball: These #EURO2020 performances 🌟 https://t.co/yO1OKcCq6g
Long: None, Lati: None
They don’t deserve you
Long: None, Lati: None
@BBCNews The England team knew that taking the knee was seen by many as a BLM gesture. 
 They could have 

KeyboardInterrupt: 